In [8]:
# import pypyodbc
import pandas as pd
from connections import connection
import pyodbc
import calendar

from datetime import datetime as dt
from datetime import timedelta as td
from datetime import date as d

import warnings


In [9]:
# Using an existing DSN (Data Source Name) for the connection


warnings.filterwarnings("ignore", category=UserWarning, message=".*only supports SQLAlchemy connectable.*")


ecrs_conn = pyodbc.connect(connection)



In [10]:

# Function to generate a list of dates for a given month and year
def generate_month_dates(year, month):
    # Find the start and end dates of the month
    start_date = dt(year, month, 1)
    
    # Get the last day of the month using calendar.monthrange
    _, last_day = calendar.monthrange(year, month)
    end_date = dt(year, month, last_day)

    # Generate the list of dates
    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime('%Y-%m-%d'))  # Store the date as a string
        current_date += td(days=1)  # Increment by 1 day

    return date_list

# Dictionary to store the dates for each month
all_month_dates = {}


# Loop through each month of 2024
for month in range(1, 13):
    all_month_dates[calendar.month_name[month]] = generate_month_dates(2022, month)

# Print the result (optional)
for month, dates in all_month_dates.items():
    print(f"{month}: {dates}")


January: ['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08', '2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15', '2022-01-16', '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28', '2022-01-29', '2022-01-30', '2022-01-31']
February: ['2022-02-01', '2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08', '2022-02-09', '2022-02-10', '2022-02-11', '2022-02-12', '2022-02-13', '2022-02-14', '2022-02-15', '2022-02-16', '2022-02-17', '2022-02-18', '2022-02-19', '2022-02-20', '2022-02-21', '2022-02-22', '2022-02-23', '2022-02-24', '2022-02-25', '2022-02-26', '2022-02-27', '2022-02-28']
March: ['2022-03-01', '2022-03-02', '2022-03-03', '2022-03-04', '2022-03-05', '2022-03-06', '2022-03-07', '2022-03-08', '2022-03-09', '2022-03-10', '2022

In [11]:
# def Month_df(date_list):
#     # Prepare a list to store DataFrames for each date range
#     df_list = []

#     # Iterate over pairs of consecutive dates using zip
#     for start_date, end_date in zip(date_list[:-1], date_list[1:]):
#         # Build the SQL query
#         ecrs_query = f"""SELECT 
#                             TRN_PK, TRN_StartTime AS StartTime, TRN_EndTime AS EndTime, TRN_ReceiptNumber AS ReceiptNumber, 
#                             CUS_AccountNum AS CustomerAccount, DPT_Name AS Department, CTI_DiscountAmount AS DiscountAmount, 
#                             TRN_Total AS TransactionTotal, TRN_NetSales AS NetSales, 
#                             CASE WHEN CTI_DiscountAmount <= 0 THEN TLI_Quantity ELSE -TLI_Quantity END AS RedemptionCount, 
#                             TLI_Quantity, TLI_Amount
#                         FROM v_TJTrans 
#                         WHERE CMB_Name LIKE '%Senior 10%' 
#                         AND TRN_EndTime BETWEEN '{start_date}' AND '{end_date}' 
#                         AND TRN_AllVoid = 0 
#                         AND ITI_VOID = 0 
#                         AND TRN_Status = 0
#         """
#         # Execute the query and append the result to df_list
#         ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)
#         df_list.append(ecrs_df)
#         # print(f'{start_date} to {end_date} appended')

#         # Handle the last date separately to ensure the full day (up to 23:59:59) is included
#     last_start_date = dt.strptime(date_list[-2], '%Y-%m-%d')
#     last_end_date = dt.strptime(date_list[-1], '%Y-%m-%d')

#     # Adjust the last_end_date to include the full day (23:59:59)
#     last_end_date = last_end_date.replace(hour=23, minute=59, second=59)

#     last_ecrs_query = f"""SELECT 
#                             TRN_PK, TRN_StartTime AS StartTime, TRN_EndTime AS EndTime, TRN_ReceiptNumber AS ReceiptNumber, 
#                             CUS_AccountNum AS CustomerAccount, DPT_Name AS Department, CTI_DiscountAmount AS DiscountAmount, 
#                             TRN_Total AS TransactionTotal, TRN_NetSales AS NetSales, 
#                             CASE WHEN CTI_DiscountAmount <= 0 THEN TLI_Quantity ELSE -TLI_Quantity END AS RedemptionCount, 
#                             TLI_Quantity, TLI_Amount
#                         FROM v_TJTrans 
#                         WHERE CMB_Name LIKE '%Senior 10%' 
#                         AND TRN_EndTime BETWEEN '{last_start_date}' AND '{last_end_date}' 
#                         AND TRN_AllVoid = 0 
#                         AND ITI_VOID = 0 
#                         AND TRN_Status = 0
#     """
#     # Execute the query for the last day and append the result
#     last_ecrs_df = pd.read_sql(last_ecrs_query, ecrs_conn)
#     df_list.append(last_ecrs_df)
#     # print(f'{last_start_date} to {last_end_date} appended')

#     # Concatenate all DataFrames for the month
#     month_df = pd.concat(df_list, ignore_index=True)
#     return month_df


In [12]:
def Month_df(date_list, ecrs_conn):
    # Prepare a list to store DataFrames for each date range
    df_list = []

    # Iterate over pairs of consecutive dates using zip
    for start_date, end_date in zip(date_list[:-1], date_list[1:]):
        # Build the SQL query
        ecrs_query = f"""SELECT 
                            TRN_PK, TRN_StartTime AS StartTime, TRN_EndTime AS EndTime, TRN_ReceiptNumber AS ReceiptNumber, 
                            CUS_AccountNum AS CustomerAccount, DPT_Name AS Department, CTI_DiscountAmount AS DiscountAmount, 
                            TRN_Total AS TransactionTotal, TRN_NetSales AS NetSales, 
                            CASE WHEN CTI_DiscountAmount <= 0 THEN TLI_Quantity ELSE -TLI_Quantity END AS RedemptionCount, 
                            TLI_Quantity, TLI_Amount
                        FROM v_TJTrans 
                        WHERE CMB_Name LIKE '%Senior 10%' 
                        AND TRN_EndTime BETWEEN '{start_date}' AND '{end_date}' 
                        AND TRN_AllVoid = 0 
                        AND ITI_VOID = 0 
                        AND TRN_Status = 0
        """
        try:
            # Execute the query and append the result to df_list
            ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)
            df_list.append(ecrs_df)
            # print(f'{start_date} to {end_date} appended')
        except Exception as e:
            print(f"Error executing query for {start_date} to {end_date}: {e}")

    # Handle the last date separately to ensure the full day (up to 23:59:59) is included
    last_start_date = dt.strptime(date_list[-2], '%Y-%m-%d')
    last_end_date = dt.strptime(date_list[-1], '%Y-%m-%d')

    # Adjust the last_end_date to include the full day (23:59:59)
    last_end_date = last_end_date.replace(hour=23, minute=59, second=59)

    last_ecrs_query = f"""SELECT 
                            TRN_PK, TRN_StartTime AS StartTime, TRN_EndTime AS EndTime, TRN_ReceiptNumber AS ReceiptNumber, 
                            CUS_AccountNum AS CustomerAccount, DPT_Name AS Department, CTI_DiscountAmount AS DiscountAmount, 
                            TRN_Total AS TransactionTotal, TRN_NetSales AS NetSales, 
                            CASE WHEN CTI_DiscountAmount <= 0 THEN TLI_Quantity ELSE -TLI_Quantity END AS RedemptionCount, 
                            TLI_Quantity, TLI_Amount
                        FROM v_TJTrans 
                        WHERE CMB_Name LIKE '%Senior 10%' 
                        AND TRN_EndTime BETWEEN '{last_start_date}' AND '{last_end_date}' 
                        AND TRN_AllVoid = 0 
                        AND ITI_VOID = 0 
                        AND TRN_Status = 0
    """
    try:
        # Execute the query for the last day and append the result
        last_ecrs_df = pd.read_sql(last_ecrs_query, ecrs_conn)
        df_list.append(last_ecrs_df)
        # print(f'{last_start_date} to {last_end_date} appended')
    except Exception as e:
        print(f"Error executing query for {last_start_date} to {last_end_date}: {e}")

    # Concatenate all DataFrames for the month
    month_df = pd.concat(df_list, ignore_index=True)
    return month_df


In [13]:
# # Define the start and end dates for April 2024
# start_date = d(2024, 4, 1)
# end_date = d(2024, 4, 30)

# # Generate the list of days in the format 'YYYY-MM-DD'
# days_in_april_2024 = [(start_date + td(days=i)).strftime('%Y-%m-%d') for i in range((end_date - start_date).days + 1)]

# # Print the result
# print(days_in_april_2024)


In [14]:
# april_df = Month_df(days_in_april_2024,ecrs_conn)

In [15]:
# Create an empty list to store all DataFrames for each month
all_month_data = []

# Iterate over all months and get the DataFrame for each
for month in all_month_dates:
    month_df = Month_df(all_month_dates[month],ecrs_conn)  # Get the DataFrame for the current month
    month_df['Month'] = f'{month} 2022'  # Add a column for the month
    month_df.to_csv(f'{month}.csv', index=False)  # Save the DataFrame to a CSV file    
    all_month_data.append(month_df)  # Append to the list
    print(f"Data for {month} appended")

# Concatenate all DataFrames into one
final_df = pd.concat(all_month_data, ignore_index=True)

# Print or return the final DataFrame
print(final_df)

C:\Users\adossantos\AppData\Local\Temp\ipykernel_18356\1108871288.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)
C:\Users\adossantos\AppData\Local\Temp\ipykernel_18356\1108871288.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)
C:\Users\adossantos\AppData\Local\Temp\ipykernel_18356\1108871288.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)
C:\Users\adossantos\AppData\Local\Tem

Data for January appended


C:\Users\adossantos\AppData\Local\Temp\ipykernel_18356\1108871288.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)
C:\Users\adossantos\AppData\Local\Temp\ipykernel_18356\1108871288.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  last_ecrs_df = pd.read_sql(last_ecrs_query, ecrs_conn)


Data for February appended


C:\Users\adossantos\AppData\Local\Temp\ipykernel_18356\1108871288.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)
C:\Users\adossantos\AppData\Local\Temp\ipykernel_18356\1108871288.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  last_ecrs_df = pd.read_sql(last_ecrs_query, ecrs_conn)


...

Data for October appended


C:\Users\adossantos\AppData\Local\Temp\ipykernel_18356\1108871288.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)
C:\Users\adossantos\AppData\Local\Temp\ipykernel_18356\1108871288.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)
C:\Users\adossantos\AppData\Local\Temp\ipykernel_18356\1108871288.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  last_ecrs_df = pd.read_sql(last_ecrs_query, ecrs_conn)
C:\Users\adossantos\AppData

Data for November appended


C:\Users\adossantos\AppData\Local\Temp\ipykernel_18356\1108871288.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)
C:\Users\adossantos\AppData\Local\Temp\ipykernel_18356\1108871288.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ecrs_df = pd.read_sql(ecrs_query, ecrs_conn)
C:\Users\adossantos\AppData\Local\Temp\ipykernel_18356\1108871288.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  last_ecrs_df = pd.read_sql(last_ecrs_query, ecrs_conn)
C:\Users\adossantos\AppData

Data for December appended
          TRN_PK               StartTime                 EndTime  \
0        1052052 2022-01-19 09:38:55.813 2022-01-19 09:41:19.542   
1        1052052 2022-01-19 09:38:55.813 2022-01-19 09:41:19.542   
2        1052052 2022-01-19 09:38:55.813 2022-01-19 09:41:19.542   
3        1052052 2022-01-19 09:38:55.813 2022-01-19 09:41:19.542   
4        1052052 2022-01-19 09:38:55.813 2022-01-19 09:41:19.542   
...          ...                     ...                     ...   
1878679  1735119 2022-12-31 10:43:24.263 2022-12-31 10:46:55.353   
1878680  1735119 2022-12-31 10:43:24.263 2022-12-31 10:46:55.353   
1878681  1735119 2022-12-31 10:43:24.263 2022-12-31 10:46:55.353   
1878682  1735119 2022-12-31 10:43:24.263 2022-12-31 10:46:55.353   
1878683  1735119 2022-12-31 10:43:24.263 2022-12-31 10:46:55.353   

        ReceiptNumber   CustomerAccount                      Department  \
0              123039  44411100001                                 None   
1     

In [16]:
final_df.head(15)



,TRN_PK,StartTime,EndTime,ReceiptNumber,CustomerAccount,Department,DiscountAmount,TransactionTotal,NetSales,RedemptionCount,TLI_Quantity,TLI_Amount,Month
0,1052052,2022-01-19 09:38:55.813,2022-01-19 09:41:19.542,123039,44411100001,None,-0.30,91.16,91.16,1.0,1.0,-0.30,January 2022
1,1052052,2022-01-19 09:38:55.813,2022-01-19 09:41:19.542,123039,44411100001,None,-0.27,91.16,91.16,1.0,1.0,-0.27,January 2022
2,1052052,2022-01-19 09:38:55.813,2022-01-19 09:41:19.542,123039,44411100001,None,-0.24,91.16,91.16,1.0,1.0,-0.24,January 2022
3,1052052,2022-01-19 09:38:55.813,2022-01-19 09:41:19.542,123039,44411100001,None,-0.24,91.16,91.16,1.0,1.0,-0.24,January 2022
4,1052052,2022-01-19 09:38:55.813,2022-01-19 09:41:19.542,123039,44411100001,None,-0.24,91.16,91.16,1.0,1.0,-0.24,January 2022
5,1052052,2022-01-19 09:38:55.813,2022-01-19 09:41:19.542,123039,44411100001,None,-0.24,91.16,91.16,1.0,1.0,-0.24,January 2022
6,1052254,2022-01-19 10:58:01.471,2022-01-19 10:59:03.537,121487,44411100001,None,-0.84,59.95,59.95,1.0,1.0,-0.84,January 2022
7,1052254,2022-01-19 10:58:01.471,2022-01-19 10:59:03.537,121487,44411100001,None,-0.74,59.95,59.95,1.0,1.0,-0.74,January 2022
8,1052254,2022-01-19 10:58:01.471,2022-01-19 10:59:03.537,121487,44411100001,None,-0.72,59.95,59.95,1.0,1.0,-0.72,January 2022
9,1052254,2022-01-19 10:58:01.471,2022-01-19 10:59:03.537,121487,44411100001,None,-0.60,59.95,59.95,1.0,1.0,-0.60,January 2022


In [17]:
# month_df.to_csv('january_2024.csv', index= False)

final_df.to_csv('monthly_2024.csv',index= False)

# don't add up all the discounts in sql, I'm missing transactions or discounts, totals are smaller than what Catapult reports